Patients -
Age, Gender

Admis

In [15]:
import pandas as pd

file_path = "./super_table.csv"  # Replace with your file path
df = pd.read_csv(file_path)


In [16]:
df = df.dropna(subset=['AdmitDiagnosis', 'religion', 'LOSgroupNum'])

df = df.drop(columns=['marital_status', 'NumChartEvents'])

df.isnull().sum()

df.loc[df['age'] > 100, 'age'] = 100

In [17]:
df.isnull().sum()

hadm_id             0
gender              0
age                 0
LOSdays             0
admit_type          0
admit_location      0
AdmitDiagnosis      0
insurance           0
religion            0
ethnicity           0
ExpiredHospital     0
NumCallouts         0
NumDiagnosis        0
NumProcs            0
NumCPTevents        0
NumInput            0
NumLabs             0
NumMicroLabs        0
NumNotes            0
NumOutput           0
NumRx               0
NumTransfers        0
TotalNumInteract    0
LOSgroupNum         0
dtype: int64

In [18]:
df['AdmitDiagnosis'].isnull().sum()

np.int64(0)

In [19]:
df.shape

(25905, 24)

In [20]:
# convert gender F to 0 and M to 1

df['gender'] = df['gender'].map({'F': 0, 'M': 1})
df.head()


,hadm_id,gender,age,LOSdays,admit_type,admit_location,AdmitDiagnosis,insurance,religion,ethnicity,...,NumCPTevents,NumInput,NumLabs,NumMicroLabs,NumNotes,NumOutput,NumRx,NumTransfers,TotalNumInteract,LOSgroupNum
0,172258,0,87,12,EMERGENCY,EMERGENCY ROOM ADMIT,PNEUMONIA,Medicare,JEWISH,WHITE,...,15,235,414,6,12,71,141,8,920,3.0
1,194619,0,0,2,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,Private,CATHOLIC,WHITE,...,0,4,32,1,9,3,0,4,421,1.0
2,113316,0,100,1,EMERGENCY,EMERGENCY ROOM ADMIT,STATUS EPILEPTIOUS,Medicare,JEWISH,WHITE,...,1,14,90,3,2,7,0,2,131,1.0
3,166489,0,0,5,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,Private,JEWISH,WHITE,...,0,0,40,1,3,0,0,4,97,2.0
4,167666,0,73,15,EMERGENCY,EMERGENCY ROOM ADMIT,"ANEMIA,THROMBOCYTOPENIA",Medicare,CHRISTIAN SCIENTIST,WHITE,...,17,2,583,25,8,3,82,4,738,4.0


In [21]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define a function to split a diagnosis, calculate embeddings for substrings, and average them
def get_mean_embedding(diagnosis):
    # Split the diagnosis into substrings
    substrings = [s.strip() for s in re.split(r'[\\,;?]', diagnosis) if s.strip()]
    
    # Get embeddings for each substring
    substring_embeddings = model.encode(substrings)
    
    # Calculate the mean embedding
    mean_embedding = np.mean(substring_embeddings, axis=0)
    
    return mean_embedding

# Apply the function to the admit_diagnosis column
import re

df['mean_embedding'] = df['AdmitDiagnosis'].apply(get_mean_embedding)

# Convert the resulting column into a 2D array for clustering
mean_embeddings = np.vstack(df['mean_embedding'].values)

In [22]:
mean_embeddings.shape

(25905, 384)

In [23]:
df['AdmitDiagnosis'].isnull().sum()

np.int64(0)

In [24]:
from sklearn.cluster import KMeans

# Define the number of clusters (you can tune this)
num_clusters = 50  # Adjust as needed based on data
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Fit and predict clusters
clusters = kmeans.fit_predict(mean_embeddings)

# Add cluster labels to the DataFrame
df['cluster'] = clusters

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
df['cluster'].isnull().sum()

np.int64(0)

In [26]:
df.head()

,hadm_id,gender,age,LOSdays,admit_type,admit_location,AdmitDiagnosis,insurance,religion,ethnicity,...,NumLabs,NumMicroLabs,NumNotes,NumOutput,NumRx,NumTransfers,TotalNumInteract,LOSgroupNum,mean_embedding,cluster
0,172258,0,87,12,EMERGENCY,EMERGENCY ROOM ADMIT,PNEUMONIA,Medicare,JEWISH,WHITE,...,414,6,12,71,141,8,920,3.0,"[-0.0001747693, 0.05518288, -0.0056504044, 0.0...",27
1,194619,0,0,2,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,Private,CATHOLIC,WHITE,...,32,1,9,3,0,4,421,1.0,"[-0.043929398, 0.051731613, -0.07686671, 0.066...",0
2,113316,0,100,1,EMERGENCY,EMERGENCY ROOM ADMIT,STATUS EPILEPTIOUS,Medicare,JEWISH,WHITE,...,90,3,2,7,0,2,131,1.0,"[0.0036525615, -0.018685985, -0.024804905, 0.0...",12
3,166489,0,0,5,NEWBORN,PHYS REFERRAL/NORMAL DELI,NEWBORN,Private,JEWISH,WHITE,...,40,1,3,0,0,4,97,2.0,"[-0.043929398, 0.051731613, -0.07686671, 0.066...",0
4,167666,0,73,15,EMERGENCY,EMERGENCY ROOM ADMIT,"ANEMIA,THROMBOCYTOPENIA",Medicare,CHRISTIAN SCIENTIST,WHITE,...,583,25,8,3,82,4,738,4.0,"[-0.06744798, 0.040656924, -0.0676014, 0.03226...",18


In [27]:
# Filter rows where cluster is 4
cluster_4_data = df[df['cluster'] == 7]

# View the diagnoses in cluster 4
diagnoses_in_cluster_4 = cluster_4_data['AdmitDiagnosis'].tolist()

# Display the first few diagnoses
print("Diagnoses in Cluster 4:")
print(diagnoses_in_cluster_4[:10])  # Display the first 10 diagnoses

Diagnoses in Cluster 4:
['RULE-OUT MYOCARDIAL INFARCTION;TELEMETRY', 'URINARY TRACT INFECTION;HYPOTENSION;TELEMETRY', 'INTRACRANIAL HEMORRHAGE;TELEMETRY', 'CORONARY ARTERY DISEASE;TELEMETRY', 'SLENIC LACERATION;TELEMETRY', 'SUBDURAL HEMATOMA;TELEMETRY', 'PULMONARY EMBOLIS;TELEMETRY', 'PNEUMONIA;TELEMETRY', 'SEPSIS;TELEMTRY', 'STROKE;TELEMETRY;TRANSIENT ISCHEMIC ATTACK']


In [28]:
df = df.drop(columns=['AdmitDiagnosis'])

In [29]:
def one_hot_encode(column_name, df):
  return pd.get_dummies(df, columns=[column_name], prefix=column_name)

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer
import re
UNIVERSAL_DELIM = "DELIM"
import re
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

UNIVERSAL_DELIM = "***"

def multi_hot_encode(column_name, df, delims=[]):
    mlb = MultiLabelBinarizer()

    # Replace delimiters with the universal delimiter
    for delim in delims:
        # Escape any special regex characters in the delimiter
        escaped_delim = re.escape(delim)
        df[column_name] = df[column_name].str.replace(escaped_delim, UNIVERSAL_DELIM, regex=True)

    # Ensure the column is a string type before splitting
    df[column_name] = df[column_name].astype(str)

    # Split the column based on the universal delimiter
    df[column_name + "_split"] = df[column_name].str.split(re.escape(UNIVERSAL_DELIM))

    # Handle any NaN or missing values by replacing them with empty lists
    df[column_name + "_split"] = df[column_name + "_split"].apply(lambda x: [] if isinstance(x, float) else x)

    # Apply MultiLabelBinarizer to the split column
    encoded = mlb.fit_transform(df[column_name + "_split"])

    # Create a DataFrame from the encoded results
    encoded_df = pd.DataFrame(encoded, columns=mlb.classes_)

    # Concatenate the encoded columns back into the original DataFrame
    df = pd.concat([df, encoded_df], axis=1)

    # Drop the original and split columns if no longer needed
    df = df.drop(columns=[column_name, column_name + "_split"])

    return df

In [31]:
df = one_hot_encode("admit_type", df)
df.head()

,hadm_id,gender,age,LOSdays,admit_location,insurance,religion,ethnicity,ExpiredHospital,NumCallouts,...,NumRx,NumTransfers,TotalNumInteract,LOSgroupNum,mean_embedding,cluster,admit_type_ELECTIVE,admit_type_EMERGENCY,admit_type_NEWBORN,admit_type_URGENT
0,172258,0,87,12,EMERGENCY ROOM ADMIT,Medicare,JEWISH,WHITE,0,0,...,141,8,920,3.0,"[-0.0001747693, 0.05518288, -0.0056504044, 0.0...",27,False,True,False,False
1,194619,0,0,2,PHYS REFERRAL/NORMAL DELI,Private,CATHOLIC,WHITE,0,0,...,0,4,421,1.0,"[-0.043929398, 0.051731613, -0.07686671, 0.066...",0,False,False,True,False
2,113316,0,100,1,EMERGENCY ROOM ADMIT,Medicare,JEWISH,WHITE,0,0,...,0,2,131,1.0,"[0.0036525615, -0.018685985, -0.024804905, 0.0...",12,False,True,False,False
3,166489,0,0,5,PHYS REFERRAL/NORMAL DELI,Private,JEWISH,WHITE,0,0,...,0,4,97,2.0,"[-0.043929398, 0.051731613, -0.07686671, 0.066...",0,False,False,True,False
4,167666,0,73,15,EMERGENCY ROOM ADMIT,Medicare,CHRISTIAN SCIENTIST,WHITE,0,0,...,82,4,738,4.0,"[-0.06744798, 0.040656924, -0.0676014, 0.03226...",18,False,True,False,False


In [32]:
df = multi_hot_encode("admit_location", df, ['/'])
df.head()

,hadm_id,gender,age,LOSdays,insurance,religion,ethnicity,ExpiredHospital,NumCallouts,NumDiagnosis,...,EMERGENCY ROOM ADMIT,EXTRAM,HMO REFERRAL,NORMAL DELI,PHYS REFERRAL,PREMATURE,SICK,TRANSFER FROM HOSP,TRANSFER FROM OTHER HEALT,TRANSFER FROM SKILLED NUR
0,172258.0,0.0,87.0,12.0,Medicare,JEWISH,WHITE,0.0,0.0,11.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,194619.0,0.0,0.0,2.0,Private,CATHOLIC,WHITE,0.0,0.0,4.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,113316.0,0.0,100.0,1.0,Medicare,JEWISH,WHITE,0.0,0.0,8.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,166489.0,0.0,0.0,5.0,Private,JEWISH,WHITE,0.0,0.0,3.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,167666.0,0.0,73.0,15.0,Medicare,CHRISTIAN SCIENTIST,WHITE,0.0,0.0,13.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df = one_hot_encode("insurance", df)
df.head()

,hadm_id,gender,age,LOSdays,religion,ethnicity,ExpiredHospital,NumCallouts,NumDiagnosis,NumProcs,...,PREMATURE,SICK,TRANSFER FROM HOSP,TRANSFER FROM OTHER HEALT,TRANSFER FROM SKILLED NUR,insurance_Government,insurance_Medicaid,insurance_Medicare,insurance_Private,insurance_Self Pay
0,172258.0,0.0,87.0,12.0,JEWISH,WHITE,0.0,0.0,11.0,7.0,...,0.0,0.0,0.0,0.0,0.0,False,False,True,False,False
1,194619.0,0.0,0.0,2.0,CATHOLIC,WHITE,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,False,False,False,True,False
2,113316.0,0.0,100.0,1.0,JEWISH,WHITE,0.0,0.0,8.0,4.0,...,0.0,0.0,0.0,0.0,0.0,False,False,True,False,False
3,166489.0,0.0,0.0,5.0,JEWISH,WHITE,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,False,False,False,True,False
4,167666.0,0.0,73.0,15.0,CHRISTIAN SCIENTIST,WHITE,0.0,0.0,13.0,1.0,...,0.0,0.0,0.0,0.0,0.0,False,False,True,False,False


In [34]:
df = one_hot_encode("religion", df)
df.head()

,hadm_id,gender,age,LOSdays,ethnicity,ExpiredHospital,NumCallouts,NumDiagnosis,NumProcs,NumCPTevents,...,religion_JEWISH,religion_LUTHERAN,religion_METHODIST,religion_MUSLIM,religion_NOT SPECIFIED,religion_OTHER,religion_PROTESTANT QUAKER,religion_ROMANIAN EAST. ORTH,religion_UNITARIAN-UNIVERSALIST,religion_UNOBTAINABLE
0,172258.0,0.0,87.0,12.0,WHITE,0.0,0.0,11.0,7.0,15.0,...,True,False,False,False,False,False,False,False,False,False
1,194619.0,0.0,0.0,2.0,WHITE,0.0,0.0,4.0,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,113316.0,0.0,100.0,1.0,WHITE,0.0,0.0,8.0,4.0,1.0,...,True,False,False,False,False,False,False,False,False,False
3,166489.0,0.0,0.0,5.0,WHITE,0.0,0.0,3.0,0.0,0.0,...,True,False,False,False,False,False,False,False,False,False
4,167666.0,0.0,73.0,15.0,WHITE,0.0,0.0,13.0,1.0,17.0,...,False,False,False,False,False,False,False,False,False,False


In [35]:
df = one_hot_encode("ethnicity", df)
df.head()

,hadm_id,gender,age,LOSdays,ExpiredHospital,NumCallouts,NumDiagnosis,NumProcs,NumCPTevents,NumInput,...,ethnicity_PATIENT DECLINED TO ANSWER,ethnicity_PORTUGUESE,ethnicity_SOUTH AMERICAN,ethnicity_UNABLE TO OBTAIN,ethnicity_UNKNOWN/NOT SPECIFIED,ethnicity_WHITE,ethnicity_WHITE - BRAZILIAN,ethnicity_WHITE - EASTERN EUROPEAN,ethnicity_WHITE - OTHER EUROPEAN,ethnicity_WHITE - RUSSIAN
0,172258.0,0.0,87.0,12.0,0.0,0.0,11.0,7.0,15.0,235.0,...,False,False,False,False,False,True,False,False,False,False
1,194619.0,0.0,0.0,2.0,0.0,0.0,4.0,1.0,0.0,4.0,...,False,False,False,False,False,True,False,False,False,False
2,113316.0,0.0,100.0,1.0,0.0,0.0,8.0,4.0,1.0,14.0,...,False,False,False,False,False,True,False,False,False,False
3,166489.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
4,167666.0,0.0,73.0,15.0,0.0,0.0,13.0,1.0,17.0,2.0,...,False,False,False,False,False,True,False,False,False,False


In [36]:
df.rename(columns={'cluster': 'AdmitDiagnosis'}, inplace=True)

In [37]:
df['AdmitDiagnosis'].isnull().sum()

np.int64(907)

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

columns = [
    'age', 'LOSdays', 'ExpiredHospital', 'NumCallouts', 'NumDiagnosis', 'NumProcs',
    'NumCPTevents', 'NumInput', 'NumLabs', 'NumMicroLabs', 'NumNotes', 'NumOutput',
    'NumRx', 'NumTransfers', 'TotalNumInteract', 'LOSgroupNum', 'AdmitDiagnosis'
]

scaler = StandardScaler()
df[columns] = scaler.fit_transform(df[columns])

scaler = MinMaxScaler()
df[columns] = scaler.fit_transform(df[columns])

In [53]:
# Calculate the number of null values in each column
null_counts = df.isnull().sum()

# Filter and print columns with null values
print("Columns with null values:")
for col, count in null_counts.items():
    if count > 0:
        print(f"{col}")

Columns with null values:
hadm_id
gender
age
LOSdays
ExpiredHospital
NumCallouts
NumDiagnosis
NumProcs
NumCPTevents
NumInput
NumLabs
NumMicroLabs
NumNotes
NumOutput
NumRx
NumTransfers
TotalNumInteract
LOSgroupNum
mean_embedding
AdmitDiagnosis
admit_type_ELECTIVE
admit_type_EMERGENCY
admit_type_NEWBORN
admit_type_URGENT
** INFO NOT AVAILABLE **
CLINIC REFERRAL
EMERGENCY ROOM ADMIT
EXTRAM
HMO REFERRAL
NORMAL DELI
PHYS REFERRAL
PREMATURE
SICK
TRANSFER FROM HOSP
TRANSFER FROM OTHER HEALT
TRANSFER FROM SKILLED NUR


In [56]:
columns_to_check = [
    "hadm_id", "gender", "age", "LOSdays", "ExpiredHospital",
    "NumCallouts", "NumDiagnosis", "NumProcs", "NumCPTevents", 
    "NumInput", "NumLabs", "NumMicroLabs", "NumNotes", "NumOutput", 
    "NumRx", "NumTransfers", "TotalNumInteract", "LOSgroupNum", 
    "mean_embedding", "AdmitDiagnosis", "admit_type_ELECTIVE", 
    "admit_type_EMERGENCY", "admit_type_NEWBORN", "admit_type_URGENT", "CLINIC REFERRAL", "EMERGENCY ROOM ADMIT", 
    "EXTRAM", "HMO REFERRAL", "NORMAL DELI", "PHYS REFERRAL", "PREMATURE", 
    "SICK", "TRANSFER FROM HOSP", "TRANSFER FROM OTHER HEALT", 
    "TRANSFER FROM SKILLED NUR"
]

# Drop rows with null values in the specified columns
df = df.dropna(subset=columns_to_check)

In [54]:
df.drop(columns=["** INFO NOT AVAILABLE **" ], inplace=True)

In [57]:
df.to_csv('preprocessed.csv', index=False)

In [58]:
df.head()

,hadm_id,gender,age,LOSdays,ExpiredHospital,NumCallouts,NumDiagnosis,NumProcs,NumCPTevents,NumInput,...,ethnicity_PATIENT DECLINED TO ANSWER,ethnicity_PORTUGUESE,ethnicity_SOUTH AMERICAN,ethnicity_UNABLE TO OBTAIN,ethnicity_UNKNOWN/NOT SPECIFIED,ethnicity_WHITE,ethnicity_WHITE - BRAZILIAN,ethnicity_WHITE - EASTERN EUROPEAN,ethnicity_WHITE - OTHER EUROPEAN,ethnicity_WHITE - RUSSIAN
0,172258.0,0.0,0.87,0.037415,0.0,0.0,0.263158,0.175,0.051370,0.005856,...,False,False,False,False,False,True,False,False,False,False
1,194619.0,0.0,0.00,0.003401,0.0,0.0,0.078947,0.025,0.000000,0.000100,...,False,False,False,False,False,True,False,False,False,False
2,113316.0,0.0,1.00,0.000000,0.0,0.0,0.184211,0.100,0.003425,0.000349,...,False,False,False,False,False,True,False,False,False,False
3,166489.0,0.0,0.00,0.013605,0.0,0.0,0.052632,0.000,0.000000,0.000000,...,False,False,False,False,False,True,False,False,False,False
4,167666.0,0.0,0.73,0.047619,0.0,0.0,0.315789,0.025,0.058219,0.000050,...,False,False,False,False,False,True,False,False,False,False


In [59]:
df.drop(columns=['hadm_id' ], inplace=True)

/var/folders/25/10nzf1851b7g1hfscm4zgtgw0000gn/T/ipykernel_57652/3951722743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['hadm_id' ], inplace=True)


In [60]:
df.isnull().sum()

gender                                0
age                                   0
LOSdays                               0
ExpiredHospital                       0
NumCallouts                           0
                                     ..
ethnicity_WHITE                       0
ethnicity_WHITE - BRAZILIAN           0
ethnicity_WHITE - EASTERN EUROPEAN    0
ethnicity_WHITE - OTHER EUROPEAN      0
ethnicity_WHITE - RUSSIAN             0
Length: 97, dtype: int64

In [61]:
df.to_csv('preprocessed.csv', index=False)